dummy implementation of talker-reasoner with openai API

In [ ]:
import json
from typing import Dict, Any, List
import openai
from openai import OpenAI

client = OpenAI(api_key=keys["OPENAI_API_KEY"])

with open("config.yaml", "r") as f:
    keys = yaml.safe_load(f)


SYSTEM_PROMPT_REASONER = """
You are a multi-step math reasoner.
Given the conversation, student profile, and question metadata:
1. Infer the student’s confusion.
2. Update their proficiency levels.
3. Solve the math problem step by step (chain of thought).
4. If details are missing to correctly solve a question, generate clarifying questions.
These clarifying questions should be simple and easy to answer, and ensure you don't make any assumption when you solve the problem.
Output exactly a JSON matching the ReasonerOutput schema.
"""

SYSTEM_PROMPT_TALKER = """
You are a conversational tutor.
Given the shared context (conversation + reasoner output):
- Use the reasoner's chain_of_thought and belief state
- Produce a friendly Socratic prompt to guide the student
Output exactly a JSON with fields: { "next_message": <string> }.
"""

# {
#   "belief_state": {...},
#   "chain_of_thought": [...],
#   "final_answer": "...",
#   "clarification_questions": [...]
# }


class TalkerReasonerPipeline:
    def __init__(self, model: str = "gpt-4o"):
        self.model = model

    def build_shared_context(self,
                             conversation: List[Dict[str, str]],
                             student_profile: Dict[str, Any],
                             question_metadata: Dict[str, Any]) -> Dict[str, Any]:
        return {
            "conversation": conversation,
            "student_profile": student_profile,
            "question_metadata": question_metadata
        }

    def call_reasoner(self, context: Dict[str, Any]) -> Dict[str, Any]:
        messages = [
            {"role": "system", "content": SYSTEM_PROMPT_REASONER},
            {"role": "user", "content": json.dumps(context)}
        ]
        response = client.chat.completions.create(model=self.model,
        # reasoning={"effort": "medium", "summary": "auto"},
        messages=messages,
        temperature=0.2)
        text = response.choices[0].message.content
        # return json.loads(text)
        return text

    def call_talker(self, context: Dict[str, Any], reasoner_out: Dict[str, Any]) -> Dict[str, Any]:
        # merge reasoner output into shared context
        shared = {**context, **{"reasoner_output": reasoner_out}}
        messages = [
            {"role": "system", "content": SYSTEM_PROMPT_TALKER},
            {"role": "user", "content": json.dumps(shared)}
        ]
        response = client.chat.completions.create(model=self.model,
        messages=messages,
        temperature=0.7)
        text = response.choices[0].message.content
        # return json.loads(text)
        return text

    def update_context(self,
                       context: Dict[str, Any],
                       reasoner_out: Dict[str, Any],
                       talker_out: Dict[str, Any]) -> Dict[str, Any]:
        # context["conversation"].append(
        #     {"role": "reasoner", "text": reasoner_out.get("chain_of_thought", [])}
        # )
        # context["conversation"].append(
        #     {"role": "talker", "text": talker_out.get("next_message", "")}    )

        # bs = reasoner_out.get("belief_state", {})
        # prof = context["student_profile"].get("proficiency", {})
        # prof.update(bs.get("proficiency_update", {}))
        # context["student_profile"]["proficiency"] = prof

        return context


TALKER_MODEL   = "gpt-4o" 
REASON_MODEL     = "o3-mini"

pipeline = TalkerReasonerPipeline(model=TALKER_MODEL)

# Initial context provided
context = pipeline.build_shared_context(
    conversation=[
        {"role":"teacher","text":"Steven, If you had 4 of something and tripled that amount, how much would you have?"},
        {"role":"student","text":"I would have 12 of something."}
    ],
    student_profile={
        "name":"Steven","grade":7,
        "proficiency":{"multiplication":0.6,"fractions":0.4}
    },
    question_metadata={
        "original_question":"If you had 4…tripled…",
        "teacher_described_confusion":"He added an extra step after solving."
    }
)

reasoner_output = pipeline.call_reasoner(context)
# print("Reasoner output:", json.dumps(reasoner_output, indent=2))
print(f"{reasoner_output=}")

talker_output = pipeline.call_talker(context, reasoner_output)
# print("Talker output:", json.dumps(talker_output, indent=2))
print(f"{talker_output=}")

# updated_context = pipeline.update_context(context, reasoner_output, talker_output)
# # print("Updated shared context:", json.dumps(updated_context, indent=2))
# print(f"{updated_context=}")



reasoner_output='```json\n{\n  "confusion_inference": "Steven seems to understand the concept of tripling a number, as he correctly calculated 4 times 3 to get 12. However, the teacher noted that he might have added an extra step, which suggests he may be overthinking or misinterpreting the problem\'s requirements.",\n  "proficiency_update": {\n    "multiplication": 0.7,\n    "fractions": 0.4\n  },\n  "solution": {\n    "step_by_step": [\n      "Start with the number 4.",\n      "To triple a number, multiply it by 3.",\n      "Calculate 4 * 3 = 12.",\n      "The result is 12."\n    ],\n    "final_answer": 12\n  },\n  "clarifying_questions": []\n}\n```'
talker_output='```json\n{\n  "next_message": "Great job, Steven! You got the right answer by tripling 4 to get 12. Just to make sure we\'re on the same page, when you hear \'triple,\' what steps do you think of? It seems like you nailed it, but sometimes it\'s helpful to check our thought process!"\n}\n```'
